In [1]:
# import necessary libraries
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pprint
import cv2

In [2]:
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import *
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [3]:
# note filepaths
PATH_TRAIN_LABELS = '../datasets/train_labels.csv'
PATH_TRAIN_RAW = '../datasets/train_images.npy'
PATH_TEST_RAW = '../datasets/test_images.npy'
PATH_TRAIN = '../datasets/train_images_cropped.npy'
PATH_TEST = '../datasets/test_images_cropped.npy'

In [ ]:
# import datasets 
train_labels = np.array(pd.read_csv(PATH_TRAIN_LABELS, delimiter=",", header=0, index_col=0))
train_images = np.load(PATH_TRAIN, encoding="latin1")
test_images = np.load(PATH_TEST, encoding="latin1")

In [4]:
# note constants
IMG_SIZE = 100
LR = 1e-3

In [5]:
# define labels
cat0 = sorted(['sink','pear','moustache','nose','skateboard','penguin','peanut','skull','panda',
        'paintbrush','nail','apple','rifle','mug','sailboat','pineapple','spoon','rabbit',
        'shovel','rollerskates','screwdriver','scorpion','rhinoceros','pool','octagon',
        'pillow','parrot','squiggle','mouth','empty','pencil'])
cat1 = {i: cat0[i] for i in range(len(cat0))}
lb = preprocessing.LabelBinarizer()
lb.fit(cat0)

# check labels
print(cat0)
print()
print(cat1)
print()
print(lb.classes_)

['apple', 'empty', 'moustache', 'mouth', 'mug', 'nail', 'nose', 'octagon', 'paintbrush', 'panda', 'parrot', 'peanut', 'pear', 'pencil', 'penguin', 'pillow', 'pineapple', 'pool', 'rabbit', 'rhinoceros', 'rifle', 'rollerskates', 'sailboat', 'scorpion', 'screwdriver', 'shovel', 'sink', 'skateboard', 'skull', 'spoon', 'squiggle']

{0: 'apple', 1: 'empty', 2: 'moustache', 3: 'mouth', 4: 'mug', 5: 'nail', 6: 'nose', 7: 'octagon', 8: 'paintbrush', 9: 'panda', 10: 'parrot', 11: 'peanut', 12: 'pear', 13: 'pencil', 14: 'penguin', 15: 'pillow', 16: 'pineapple', 17: 'pool', 18: 'rabbit', 19: 'rhinoceros', 20: 'rifle', 21: 'rollerskates', 22: 'sailboat', 23: 'scorpion', 24: 'screwdriver', 25: 'shovel', 26: 'sink', 27: 'skateboard', 28: 'skull', 29: 'spoon', 30: 'squiggle'}

['apple' 'empty' 'moustache' 'mouth' 'mug' 'nail' 'nose' 'octagon'
 'paintbrush' 'panda' 'parrot' 'peanut' 'pear' 'pencil' 'penguin' 'pillow'
 'pineapple' 'pool' 'rabbit' 'rhinoceros' 'rifle' 'rollerskates'
 'sailboat' 'scorpion